In [1]:
import pickle
import pandas as pd
import numpy as np
import nrrd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output

from radiomics import featureextractor, getTestCase
import SimpleITK as sitk
import subprocess

import os
import glob
import shutil

import time
import datetime

print(datetime.datetime.now())

2022-02-22 18:45:14.906627


In [2]:
path_data = '../data/d20220201/'

# INPUTS
fn_patient = '../metadata/Celine_FINAL_RADIOPOP_DATA_with LRE Death_220126.xlsx'
fn_patient2 = '../metadata/List Celine Radipop 20.12.21.xlsx'
path_nrrds = path_data + 'nrrds_matched2mask/'
path_liverMasks = path_data + 'nrrds_maskLiver/'
path_spleenMasks = path_data + 'nrrds_maskSpleen/'
fn_extractorYaml = '../exampleCT.yaml'

# 
path_features = path_data + 'features/'

In [3]:
# PARAMETERS
#binWidth = 0.02   # for radiomics feature extractor, from Thomas
extractor = featureextractor.RadiomicsFeatureExtractor(fn_extractorYaml)

try:
    os.mkdir(path_features)
except OSError:
    print ("Creation of the directory %s failed" % path_features)
else:
    print ("Successfully created the directory %s " % path_features)

Creation of the directory data/d20220201/features/ failed


In [4]:
patients_df = pd.read_excel(fn_patient, index_col=0, dtype=str)#, index_col=0, header=1, nrows=371, dtype=str)
patients_df['patientID'] = patients_df.index
patients_df['patientID'] = patients_df['patientID'].apply(lambda x: str(x).zfill(4))
patients_df = patients_df.set_index('patientID')
patients_df

patients_df2 = pd.read_excel(fn_patient2, index_col=0, dtype=str)#, index_col=0, header=1, nrows=371, dtype=str)
patients_df2['temp'] = patients_df2.index
patients_df2['patientID'] = patients_df2['temp'].str.split(" ", n=1).str[0]
patients_df2.patientID.fillna(patients_df2.temp, inplace=True)
patients_df2 = patients_df2[patients_df2['patientID'].astype(int) >= 1000]
patients_df2['patientID'] = patients_df2['patientID'].astype(str)
patients_df2 = patients_df2.set_index('patientID')
patients_df2

listPatients = patients_df.index.tolist() + patients_df2.index.tolist()
print(len(listPatients))

562


In [5]:
t0 = time.time()

#listFiles = [x for x in os.listdir(path_nrrds) if x.endswith('.nrrd')]
failedIDs = {}
success = []

for eaFile in listPatients:
    t1 = time.time()
    fn_dict = path_features + eaFile + '.p'
    if os.path.exists(fn_dict):
        print(eaFile + ': features already extracted')
        success.append(eaFile)
    else:
        try:
            d_features = {}
            # get files
            fn_currRaw = path_nrrds + eaFile + '.nrrd'
            fn_liverMask = path_liverMasks + eaFile + '.nrrd'
            fn_spleenMask = path_spleenMasks + eaFile + '.nrrd'
            # extract liver features
            feat_liver = extractor.execute(fn_currRaw, fn_liverMask)
            d_features['liver'] = feat_liver
            # extract spleen features
            feat_spleen = extractor.execute(fn_currRaw, fn_spleenMask)
            d_features['spleen'] = feat_spleen
            print(str(eaFile) + ' features extracted: ' + str(time.time() - t1) + ' seconds.')
            with open(fn_dict, 'wb') as handle:
                pickle.dump(d_features, handle, protocol=pickle.HIGHEST_PROTOCOL)
            success.append(eaFile)
        except Exception as e:
            print(eaFile + ' failed extraction.')
            failedIDs[eaFile] = e

print(time.time() - t0)
print(datetime.datetime.now())

0001: features already extracted
0002: features already extracted
0004: features already extracted
0005: features already extracted
0006: features already extracted
0007: features already extracted
0008: features already extracted
0009: features already extracted
0010: features already extracted
0011: features already extracted
0013: features already extracted
0015: features already extracted
0016: features already extracted
0018: features already extracted
0019: features already extracted
0020: features already extracted
0021: features already extracted
0022: features already extracted
0023: features already extracted
0024: features already extracted
0025: features already extracted
0026: features already extracted
0027: features already extracted
0028: features already extracted
0029: features already extracted
0030: features already extracted
0031: features already extracted
0032: features already extracted
0033: features already extracted
0034: features already extracted
0035: feat

0311 features extracted: 162.74614453315735 seconds.
0312: features already extracted
0313: features already extracted
0314: features already extracted
0317: features already extracted
0318: features already extracted
0319: features already extracted
0321: features already extracted
0322: features already extracted
0323: features already extracted
0324: features already extracted
0325: features already extracted
0326: features already extracted
0327: features already extracted
0330: features already extracted
0331: features already extracted
0332: features already extracted
0333: features already extracted
0334: features already extracted
0335: features already extracted
0336: features already extracted
0337: features already extracted
0338: features already extracted
0339: features already extracted
0340: features already extracted
0341: features already extracted
0342: features already extracted
0343: features already extracted
0346: features already extracted
0347: features already 

In [6]:
print(success)

['0001', '0002', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0013', '0015', '0016', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030', '0031', '0032', '0033', '0034', '0035', '0036', '0037', '0038', '0039', '0040', '0041', '0042', '0043', '0044', '0045', '0046', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059', '0060', '0061', '0062', '0063', '0064', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0076', '0077', '0078', '0079', '0080', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0088', '0089', '0091', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0107', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0117', '0118', '0119', '0120', '0121', '0122', '0123', '0124', '0125', '0126', '0127', '0128', '0130', '0131', '0132', '0133', '0134', '0135',

In [7]:
print(failedIDs.keys())

dict_keys([])


In [8]:
print(failedIDs)

{}


In [9]:
print(datetime.datetime.now())

2022-02-22 18:48:00.506449
